Examine the data structure in our raster files and shp files.

In [6]:
import pathlib
import rasterio
import fiona
import pandas as pd
from shapely.geometry import shape # Useful for more advanced geometry analysis
from collections import defaultdict

# --- Configuration ---
root_dir = pathlib.Path("/Users/faizahmad/Desktop/grok_live/data")  # 📂 Adjust this path

# --- Function to Extract Metadata from TIF (Raster) Files (from previous version, unchanged) ---
def extract_tif_metadata(directory: pathlib.Path) -> pd.DataFrame:
    """
    Extracts common metadata from GeoTIFF files in a given directory.
    """
    tif_files = sorted(directory.glob("*.tif"))
    records = []
    for f in tif_files:
        try:
            with rasterio.open(f) as src:
                meta = src.meta
                records.append({
                    "file": f.name,
                    "type": "raster",
                    "crs": meta["crs"].to_string() if meta["crs"] else None,
                    "width": meta["width"],
                    "height": meta["height"],
                    "transform": tuple(round(x, 10) for x in meta["transform"][:6]),
                    "dtype": meta["dtype"],
                    "nodata": meta["nodata"],
                    "bands": meta["count"],
                    "resolution_x": round(meta["transform"].a, 10),
                    "resolution_y": round(meta["transform"].e, 10),
                    "driver": meta["driver"],
                })
        except rasterio.errors.RasterioIOError as e:
            print(f"⚠️  Skipping {f.name}: Not a valid GeoTIFF or read error: {e}")
        except Exception as e:
            print(f"⚠️  An unexpected error occurred with {f.name}: {e}")
    return pd.DataFrame(records)

# --- ❷ Further Improved Function to Extract Metadata from SHP (Vector) Files ---
def extract_shp_metadata(directory: pathlib.Path) -> pd.DataFrame:
    """
    Extracts common metadata from Shapefile files in a given directory.
    """
    shp_files = sorted(directory.glob("*.shp"))
    records = []
    for f in shp_files:
        try:
            with fiona.open(f) as src:
                # --- Robustly extract properties schema ---
                properties_schema = {}
                if 'properties' in src.schema and src.schema['properties']:
                    for prop_name, prop_details in src.schema["properties"].items():
                        if isinstance(prop_details, dict) and 'type' in prop_details:
                            properties_schema[prop_name] = prop_details['type']
                        else:
                            # Fallback if 'type' key is missing or prop_details is not a dict
                            properties_schema[prop_name] = str(prop_details) # Coerce to string for inspection
                else:
                    properties_schema = {} # No properties found or empty schema

                # --- Get unique geometry types (sampling for performance) ---
                unique_geometry_types = set()
                try:
                    for i, feature in enumerate(src):
                        if i >= 1000: # Sample up to 1000 features to avoid slow processing
                            break
                        if feature and 'geometry' in feature and feature['geometry'] and 'type' in feature['geometry']:
                            unique_geometry_types.add(feature['geometry']['type'])
                except StopIteration:
                    pass # Handle empty shapefiles gracefully
                except Exception as geom_e:
                    print(f"⚠️  Error getting geometry type for {f.name}: {geom_e}")
                    unique_geometry_types.add("Error_reading_geometry")


                records.append({
                    "file": f.name,
                    "type": "vector",
                    "crs": src.crs_wkt,  # WKT string for CRS
                    "geometry_types": list(unique_geometry_types) if unique_geometry_types else [], # List of detected geometry types
                    "properties_schema": properties_schema, # Field names and types
                    "bounds": src.bounds, # (min_x, min_y, max_x, max_y)
                    "driver": src.driver,
                    "feature_count": len(src), # Number of features in the shapefile
                    "encoding": src.encoding if hasattr(src, 'encoding') else None, # Character encoding for DBF
                })
        except fiona.errors.DriverError as e:
            print(f"⚠️  Skipping {f.name}: Not a valid Shapefile or read error: {e}")
        except Exception as e:
            print(f"⚠️  An unexpected error occurred with {f.name}: {e}")
            # This line will help diagnose the exact structure if the above fixes don't work
            # if 'src' in locals():
            #     print(f"Debug: Schema for {f.name}: {src.schema}")
            #     if 'properties' in src.schema:
            #         print(f"Debug: Properties for {f.name}: {src.schema['properties']}")
    return pd.DataFrame(records)

# --- ❸ Run Extraction and Analysis ---

print("--- Extracting TIF Metadata ---")
df_tifs = extract_tif_metadata(root_dir)
if not df_tifs.empty:
    print(df_tifs.to_string()) # Use .to_string() to avoid truncation
    print("\n⚠️  Inconsistent fields for TIFs:", end=" ")
    tif_problems = {
        col: df_tifs[col].unique().tolist()
        for col in ["crs", "width", "height", "transform", "dtype", "nodata", "bands", "resolution_x", "resolution_y", "driver"]
        if len(df_tifs[col].unique()) > 1
    }
    print(tif_problems if tif_problems else "None 🎉")
else:
    print("No TIF files found or processed.")


print("\n--- Extracting SHP Metadata ---")
df_shps = extract_shp_metadata(root_dir)
if not df_shps.empty:
    print(df_shps.to_string()) # Use .to_string() to avoid truncation

    print("\n⚠️  Inconsistent fields for SHPs:", end=" ")
    # For SHP consistency, 'bounds' and 'feature_count' are almost certainly
    # going to be different for each file, so exclude them from the default check.

    # Convert 'properties_schema' dictionary to a sorted string for hashing
    properties_schema_str = df_shps['properties_schema'].apply(lambda x: str(sorted(x.items())))
    geometry_types_hashable = df_shps['geometry_types'].apply(lambda x: tuple(sorted(x)))

    shp_problems = {
        "crs": df_shps["crs"].unique().tolist() if len(df_shps["crs"].unique()) > 1 else None,
        "geometry_types": geometry_types_hashable.unique().tolist() if len(geometry_types_hashable.unique()) > 1 else None,
        "properties_schema": properties_schema_str.unique().tolist() if len(properties_schema_str.unique()) > 1 else None,
        "driver": df_shps["driver"].unique().tolist() if len(df_shps["driver"].unique()) > 1 else None,
        "encoding": df_shps["encoding"].unique().tolist() if len(df_shps["encoding"].unique()) > 1 else None,
    }
    # Remove None values
    shp_problems = {k: v for k, v in shp_problems.items() if v is not None}
    print(shp_problems if shp_problems else "None 🎉")

    # Optional: More specific check for schema consistency, if fields should be identical
    if len(properties_schema_str.unique()) > 1:
        print("\n💡 Note: 'properties_schema' field consistency check shows differences in field names or types.")
        # You could add logic here to print the differing schemas for inspection
else:
    print("No SHP files found or processed.")

# --- Optional: Combine and Analyze if you need a single DataFrame ---
# You can concatenate them if you add a 'type' column to distinguish
# combined_df = pd.concat([df_tifs, df_shps], ignore_index=True)
# print("\n--- Combined Metadata (if applicable) ---")
# print(combined_df)

--- Extracting TIF Metadata ---
                      file    type        crs  width  height                                              transform    dtype         nodata  bands  resolution_x  resolution_y driver
0         5_Bf_2010_Da.tif  raster  EPSG:4326   4320    2160  (0.0833333333, 0.0, -180.0, 0.0, -0.0833333333, 90.0)  float64 -1.700000e+308      1      0.083333     -0.083333  GTiff
1         5_Bf_2015_Da.tif  raster  EPSG:4326   4320    2160  (0.0833333333, 0.0, -180.0, 0.0, -0.0833333333, 90.0)  float32  -3.400000e+38      1      0.083333     -0.083333  GTiff
2         5_Ch_2010_Da.tif  raster  EPSG:4326   4320    2160  (0.0833333333, 0.0, -180.0, 0.0, -0.0833333333, 90.0)  float64 -1.700000e+308      1      0.083333     -0.083333  GTiff
3         5_Ch_2015_Da.tif  raster  EPSG:4326   4320    2160  (0.0833333333, 0.0, -180.0, 0.0, -0.0833333333, 90.0)  float32  -3.400000e+38      1      0.083333     -0.083333  GTiff
4         5_Ct_2010_Da.tif  raster  EPSG:4326   4320    21

For the livestock dataset: Calculating mean, count and headcount 

In [ ]:
import os, re
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
import geopandas as gpd
import pandas as pd
from rasterstats import zonal_stats
from tqdm import tqdm

RASTER_DIR = Path("/Users/faizahmad/Desktop/NewLiv/data")
COUNTY_SHP = Path("/Users/faizahmad/Desktop/NewLiv/data/cb_2023_us_county_500k.shp")
OUT_CSV    = Path("county_livestock_2010_2015_2020.csv")
CORES      = max(os.cpu_count() - 1, 1)

def load_counties():
    g = gpd.read_file(COUNTY_SHP).to_crs("EPSG:4326")
    geoids = [c for c in g.columns if c.upper().startswith("GEOID")]
    col = "GEOID" if "GEOID" in geoids else sorted(geoids, key=len)[0]
    if col != "GEOID":
        g = g.rename(columns={col: "GEOID"})
    if "ALAND" in g.columns:
        g["area_km2"] = g["ALAND"] / 1e6
    else:
        g["area_km2"] = g.to_crs(5070).geometry.area / 1e6
    return g[["GEOID","area_km2","geometry"]]

def one_raster(tif, counties):
    year = re.search(r"(\d{4})", tif.name).group(1)
    species = tif.stem.split('.')[-1]
    stats = zonal_stats(
        counties, tif,
        stats=["mean","count"],
        nodata=None,
        all_touched=True
    )
    df = pd.DataFrame(stats)
    df["GEOID"] = counties["GEOID"].values
    df["year"]  = year
    df["species"] = species
    df["head_count"] = df["mean"] * counties["area_km2"]
    return df[["GEOID","year","species","head_count","mean","count"]].rename(
        columns={"mean":"mean_density","count":"cell_count"}
    )

def run_batch():
    counties = load_counties()
    tifs = sorted(RASTER_DIR.glob("*.tif"))
    pieces = []
    with ThreadPoolExecutor(max_workers=CORES) as ex:
        futs = {ex.submit(one_raster, tif, counties): tif for tif in tifs}
        for f in tqdm(as_completed(futs), total=len(futs), desc="processing"):
            pieces.append(f.result())
    out = pd.concat(pieces, ignore_index=True)
    out.to_csv(OUT_CSV, index=False)
    return out

df = run_batch()


For the livestock datset core code that ensures robustness in the process to use NLCD asset to facillitate our computation (Using NLCD Agri Land "weighted_mean": weighted_mean, "std_dev": std_dev, "std_error": std_error, "head_count": head_count, "area_usable_km2": usable_area_km2)

In [ ]:
! pip install rasterio

import os
import re
from pathlib import Path
import geopandas as gpd
import pandas as pd
import numpy as np
import rasterio
from rasterio.mask import mask as rio_mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.windows import from_bounds, intersection
from tqdm import tqdm
import gc
from concurrent.futures import ProcessPoolExecutor, as_completed
from math import radians, cos

# Configuration
DATA_DIR = Path("/content/drive/MyDrive/NewLiv/data")
COUNTY_SHP = DATA_DIR / "cb_2023_us_county_500k.shp"
NLCD_RASTER = DATA_DIR / "Annual_NLCD_LndCov_2019_CU_C1V1.tif"
OUT_DIR = DATA_DIR / "temp_county_stats"
OUT_DIR.mkdir(exist_ok=True)
CORES = os.cpu_count() or 1
COMBINED_OUT = DATA_DIR / "combined_livestock_stats.csv"

# Land cover classes (NLCD codes) based on metadata
LAND_USE_CLASSES = {
    "cropland": [82],  # Cultivated Crops
    "grassland": [71],  # Grassland/Herbaceous
    "pasture": [81],   # Pasture/Hay
    "permanent_crops": [],
    "water_bodies": [11, 12],
    "high_pop": [24],
    "other_exclusions": [21, 22, 23, 31, 52, 90, 95]
}

# Species mapping
code2name = {
    "CH": "chicken", "CHK": "chicken", "PG": "pig", "PGS": "pig",
    "CT": "cattle", "CTL": "cattle", "GT": "goat", "GTS": "goat",
    "SH": "sheep", "SHP": "sheep", "BF": "buffalo", "BFL": "buffalo",
    "DK": "duck", "HO": "horse"
}

# Helper Functions
def load_counties():
    """Load and prepare county shapefile."""
    g = gpd.read_file(COUNTY_SHP)
    geoids = [c for c in g.columns if c.upper().startswith("GEOID")]
    col = "GEOID" if "GEOID" in geoids else sorted(geoids, key=len)[0]
    if col != "GEOID":
        g = g.rename(columns={col: "GEOID"})
    g["GEOID"] = g["GEOID"].str.zfill(5)
    g["area_km2"] = g["ALAND"] / 1e6 if "ALAND" in g.columns else g.to_crs(5070).geometry.area / 1e6
    g = g.to_crs("EPSG:4326")
    return g[["GEOID", "area_km2", "geometry"]]

def compute_weighted_mean(data, nodata, geometry, transform, county_area_km2):
    """Compute weighted mean density, standard deviation, and standard error."""
    valid_data = data[data != nodata]
    if len(valid_data) == 0:
        return np.nan, np.nan, np.nan
    # Use accurate cell area based on latitude
    centroid_lat = geometry.centroid.y
    cell_area = (transform.a * 111.32 * np.cos(np.radians(centroid_lat))) * (abs(transform.e) * 111.32)
    weights = np.ones_like(valid_data) * cell_area
    weighted_mean = np.average(valid_data, weights=weights) if len(valid_data) > 0 else np.nan
    std_dev = np.std(valid_data) if len(valid_data) > 0 else np.nan
    std_error = std_dev / np.sqrt(len(valid_data)) if len(valid_data) > 0 else np.nan
    return weighted_mean, std_dev, std_error

def bounds_overlap(bounds1, bounds2):
    """Check if two bounding boxes overlap."""
    return not (bounds1[0] > bounds2[2] or bounds1[2] < bounds2[0] or bounds1[1] > bounds2[3] or bounds1[3] < bounds2[1])

def extract_species_from_filename(filename):
    """Extract species code from raster filename based on known patterns."""
    match_glw4 = re.search(r"GLW4-\d{4}\.D-DA\.([A-Z]{3})\.tif", filename)
    if match_glw4:
        return match_glw4.group(1)
    match_5_da = re.search(r"5_([A-Za-z]{2})_\d{4}_Da\.tif", filename)
    if match_5_da:
        return match_5_da.group(1).upper()
    return ""

def compute_idw_density(geoid, species, year, counties, livestock_raster_paths):
    """Compute IDW-based density for counties with zero agricultural area."""
    county = counties[counties["GEOID"] == geoid].iloc[0]
    centroid = county.geometry.centroid
    counties_proj = counties.to_crs("EPSG:5070")
    county_proj = gpd.GeoSeries([centroid], crs="EPSG:4326").to_crs("EPSG:5070").iloc[0]
    neighbors = counties_proj[counties_proj.geometry.distance(county_proj) < 11000]
    weights = []
    densities = []
    for _, neighbor in neighbors.iterrows():
        if neighbor["GEOID"] == geoid:
            continue
        neighbor_csv = OUT_DIR / f"county_{neighbor['GEOID']}_stats.csv"
        if neighbor_csv.exists():
            df = pd.read_csv(neighbor_csv)
            density_val = df[(df["year"] == year) & (df["species"] == species)]["weighted_mean"]
            if not density_val.empty and not np.isnan(density_val.iloc[0]):
                dist = county_proj.distance(neighbor.geometry)
                if dist > 0:
                    weights.append(1 / dist)
                    densities.append(density_val.iloc[0])
    if weights:
        return np.average(densities, weights=weights)
    return np.nan

def resample_nlcd_to_livestock(nlcd_src, livestock_src):
    """Resample NLCD raster to match livestock raster resolution."""
    transform, width, height = calculate_default_transform(
        nlcd_src.crs, livestock_src.crs, livestock_src.width, livestock_src.height,
        *nlcd_src.bounds
    )
    nlcd_resampled = np.zeros((livestock_src.height, livestock_src.width), dtype=nlcd_src.dtypes[0])
    reproject(
        source=rasterio.band(nlcd_src, 1),
        destination=nlcd_resampled,
        src_transform=nlcd_src.transform,
        src_crs=nlcd_src.crs,
        dst_transform=transform,
        dst_crs=livestock_src.crs,
        resampling=Resampling.nearest
    )
    return nlcd_resampled, transform

def process_county_chunk(county_chunk, nlcd_raster_path, livestock_raster_paths, counties):
    """Process a chunk of counties with optimized raster handling."""
    results = []

    # --- START of efficiency fix ---
    # Perform expensive raster operations once per chunk.
    # The CRS is consistently EPSG:4326 for all rasters, making this simpler.
    with rasterio.open(nlcd_raster_path) as nlcd_src:
        with rasterio.open(livestock_raster_paths[0]) as liv_src:
            # Check for CRS consistency before resampling
            if nlcd_src.crs != liv_src.crs:
                print(f"Warning: NLCD ({nlcd_src.crs}) and livestock ({liv_src.crs}) CRSs don't match. Reprojecting NLCD.")

            nlcd_resampled, nlcd_transform_resampled = resample_nlcd_to_livestock(nlcd_src, liv_src)
            nlcd_bounds = nlcd_src.bounds

    # --- END of efficiency fix ---

    for _, county in county_chunk.iterrows():
        gc.collect()
        geoid = county["GEOID"]
        geometry_4326 = county["geometry"]

        # Use the CRS from the resampled NLCD/livestock raster for consistency.
        county_geom_proj = gpd.GeoSeries([geometry_4326], crs="EPSG:4326").to_crs(rasterio.open(livestock_raster_paths[0]).crs).iloc[0]
        county_bounds_proj = county_geom_proj.bounds

        usable_area_km2 = 0.0

        if bounds_overlap(county_bounds_proj, nlcd_bounds):
            try:
                # Use the resampled NLCD data and its transform
                county_window = from_bounds(*county_bounds_proj, nlcd_transform_resampled)
                clamped_window = intersection(county_window, from_bounds(*nlcd_bounds, nlcd_transform_resampled))

                if clamped_window.width > 0 and clamped_window.height > 0:
                    chunk = nlcd_resampled[
                        int(clamped_window.row_off):int(clamped_window.row_off + clamped_window.height),
                        int(clamped_window.col_off):int(clamped_window.col_off + clamped_window.width)
                    ]

                    centroid_lat = county_geom_proj.centroid.y
                    # Correct cell area calculation
                    cell_area_km2 = (nlcd_transform_resampled.a * 111.32 * np.cos(np.radians(centroid_lat))) * (abs(nlcd_transform_resampled.e) * 111.32)

                    exclude_mask = (
                        np.isin(chunk, LAND_USE_CLASSES["water_bodies"]) |
                        np.isin(chunk, LAND_USE_CLASSES["high_pop"]) |
                        np.isin(chunk, LAND_USE_CLASSES["other_exclusions"])
                    )

                    agri_cells = (
                        np.isin(chunk, LAND_USE_CLASSES["cropland"]) |
                        np.isin(chunk, LAND_USE_CLASSES["grassland"]) |
                        np.isin(chunk, LAND_USE_CLASSES["pasture"])
                    ) & ~exclude_mask

                    usable_area_km2 = np.sum(agri_cells) * cell_area_km2

                    # More robust check for usable area
                    if usable_area_km2 < 0.1 or usable_area_km2 > county["area_km2"] * 1.5:
                        print(f"Warning: Unusual usable area ({usable_area_km2:.2f} km²) for county {geoid}, using full county area.")
                        usable_area_km2 = county["area_km2"]

            except (ValueError, rasterio.errors.RasterioIOError) as e:
                print(f"Error processing NLCD for county {geoid}: {e}. Falling back to total area.")
                usable_area_km2 = county["area_km2"]
        else:
            usable_area_km2 = county["area_km2"]

        stats = []
        for tif_path in livestock_raster_paths:
            gc.collect()
            with rasterio.open(tif_path) as src:
                year = re.search(r"(\d{4})", Path(tif_path).name).group(1)
                species_code = extract_species_from_filename(Path(tif_path).name)
                species = code2name.get(species_code, species_code.lower())
                nodata = src.nodata if src.nodata is not None else np.nan

                # Check for overlap and process
                weighted_mean, std_dev, std_error = np.nan, np.nan, np.nan
                if bounds_overlap(county_bounds_proj, src.bounds):
                    try:
                        # Use rio_mask for more accurate clipping
                        out_image, out_transform = rio_mask(src, [county_geom_proj], crop=True, all_touched=True)
                        if out_image.size > 0:
                            weighted_mean, std_dev, std_error = compute_weighted_mean(
                                out_image[0], nodata, county_geom_proj, out_transform, county["area_km2"]
                            )
                    except (ValueError, rasterio.errors.RasterioIOError) as e:
                        print(f"Error processing {species} raster for county {geoid}: {e}")
                        weighted_mean = np.nan

                # Handle zero agricultural area with IDW
                if usable_area_km2 < 0.1 and not np.isnan(weighted_mean):
                    weighted_mean = compute_idw_density(geoid, species, year, counties, livestock_raster_paths)
                    std_dev, std_error = np.nan, np.nan

                area_for_calc = usable_area_km2 if usable_area_km2 > 0.1 else county["area_km2"]
                head_count = weighted_mean * area_for_calc if not np.isnan(weighted_mean) else np.nan

                stats.append({
                    "GEOID": geoid, "year": year, "species": species,
                    "weighted_mean": weighted_mean, "std_dev": std_dev, "std_error": std_error,
                    "head_count": head_count, "area_usable_km2": usable_area_km2
                })

        df = pd.DataFrame(stats)
        out_path = OUT_DIR / f"county_{geoid}_stats.csv"
        df.to_csv(out_path, index=False)
        results.append(out_path)
    return results

def combine_csvs(out_paths):
    """Combine all county CSV files into a single CSV."""
    if not out_paths:
        print("No county stats files were generated to combine.")
        return
    combined_df = pd.concat([pd.read_csv(f) for f in out_paths], ignore_index=True)
    combined_df.to_csv(COMBINED_OUT, index=False)
    print(f"\n✅ Combined CSV saved to: {COMBINED_OUT}")

def main_processing(livestock_raster_paths, counties, chunk_size=5):
    """Orchestrate parallel processing of all counties."""
    out_paths = []
    county_chunks = [counties[i:i + chunk_size] for i in range(0, len(counties), chunk_size)]

    with ProcessPoolExecutor(max_workers=CORES) as executor:
        futures = {
            executor.submit(
                process_county_chunk, chunk, NLCD_RASTER, livestock_raster_paths, counties
            ): chunk for chunk in county_chunks
        }
        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing chunks"):
            out_paths.extend(future.result())

    print(f"\n✅ Saved {len(out_paths)} county CSVs to: {OUT_DIR}")
    combine_csvs(out_paths)
    return out_paths

if __name__ == "__main__":
    print("Loading counties...")
    counties = load_counties()
    print("Finding raster file paths...")
    livestock_raster_paths = [
        str(f) for f in sorted(DATA_DIR.glob("*.tif"))
        if "NLCD" not in f.name and "pixel_area" not in f.name
    ]
    if not livestock_raster_paths:
        print("No livestock raster files found. Exiting.")
    else:
        print(f"Found {len(livestock_raster_paths)} livestock rasters.")
        chunk_size = CORES * 2
        print(f"Using a chunk size of {chunk_size} counties.")
        main_processing(livestock_raster_paths, counties, chunk_size)

Feature Enginnering - core code that creates engineered features 

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime

# Configuration
DATA_DIR = Path("/content/drive/MyDrive/NewLiv/data")
input_file = DATA_DIR / "interpolated_livestock_wide.csv"
output_file = DATA_DIR / "interpolated_livestock_wide_engineered.csv"

# Load the wide-format CSV
df = pd.read_csv(input_file, na_values=['', 'NA', '-'])

def engineer_features_fast(df):
    # Dynamically extract species from column names
    area_cols = [col for col in df.columns if col.startswith('area_usable_km2_')]
    species = [col.replace('area_usable_km2_', '') for col in area_cols]

    # Compute usable_area_percent for each species (vectorized)
    # The np.where logic correctly handles division by zero.
    for species_name in species:
        area_col = f'area_usable_km2_{species_name}'
        percent_col = f'usable_area_percent_{species_name}'
        if area_col in df.columns and 'total_area_km2' in df.columns:
            df[percent_col] = np.where(df['total_area_km2'] != 0, (df[area_col] / df['total_area_km2']) * 100, np.nan)

    # Compute growth rates for weighted_mean and head_count (vectorized)
    # This replaces the slow loops with a groupby and vectorized pct_change.
    df = df.sort_values(by=['GEOID', 'year']).set_index(['GEOID', 'year'])

    for species_name in species:
        wm_col = f'weighted_mean_{species_name}'
        hc_col = f'head_count_{species_name}'

        # Calculate percent change grouped by GEOID
        if wm_col in df.columns:
            df[f'growth_rate_wm_{species_name}'] = df.groupby(level='GEOID')[wm_col].pct_change().mul(100)

        if hc_col in df.columns:
            df[f'growth_rate_hc_{species_name}'] = df.groupby(level='GEOID')[hc_col].pct_change().mul(100)

    # Compute aggregated features (vectorized)
    # Sum and mean are already vectorized functions in pandas.
    wm_cols = [f'weighted_mean_{s}' for s in species if f'weighted_mean_{s}' in df.columns]
    hc_cols = [f'head_count_{s}' for s in species if f'head_count_{s}' in df.columns]

    if wm_cols:
        df['total_weighted_mean'] = df[wm_cols].sum(axis=1)
        df['avg_weighted_mean'] = df[wm_cols].mean(axis=1)

    if hc_cols:
        df['total_head_count'] = df[hc_cols].sum(axis=1)

    return df.reset_index()

# Apply the fast feature engineering
engineered_df = engineer_features_fast(df.copy()) # Use a copy to avoid chained assignment warnings

# Save the result
engineered_df.to_csv(output_file, index=False)
print(f"Engineered features saved to {output_file} at {datetime.now().strftime('%I:%M %p CDT on %B %d, %Y')}")

Livestock datset Interpolation code - using the datasets from 2010, 2015, 2020, we create a dataset to match our LE temporal resolution 

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
import geopandas as gpd

# Configuration
DATA_DIR = Path("/content/drive/MyDrive/NewLiv/data")
input_file = DATA_DIR / "combined_livestock_stats.csv"
output_file = DATA_DIR / "interpolated_livestock_wide.csv"
county_shp = DATA_DIR / "cb_2023_us_county_500k.shp"

# Load county data for total area
counties_gdf = gpd.read_file(county_shp)
counties_gdf = counties_gdf[["GEOID", "ALAND", "AWATER"]].copy()

# --- FIX: Standardize GEOID in shapefile data to a 5-digit string ---
counties_gdf["GEOID"] = counties_gdf["GEOID"].astype(str).str.zfill(5)

counties_gdf["total_area_km2"] = (counties_gdf["ALAND"] + counties_gdf["AWATER"]) / 1e6
counties_gdf = counties_gdf.drop(columns=["ALAND", "AWATER"])
county_area_map = counties_gdf.set_index('GEOID')['total_area_km2'].to_dict()

# Load the combined CSV
df = pd.read_csv(input_file, na_values=['', 'NA', '-'])

# --- FIX: Standardize GEOID in the CSV data to a 5-digit string ---
df["GEOID"] = df["GEOID"].astype(str).str.zfill(5)

# Define the year range for interpolation
start_year = 2003
end_year = 2020
years = range(start_year, end_year + 1)

# Function to interpolate values for a given GEOID and species
def interpolate_group(group):
    full_index = pd.Index(years, name='year')
    full_df = group.set_index('year').reindex(full_index)

    full_df['GEOID'] = group['GEOID'].iloc[0]
    full_df['species'] = group['species'].iloc[0]

    if 'area_usable_km2' in group.columns and not group['area_usable_km2'].empty:
        full_df['area_usable_km2'] = group['area_usable_km2'].iloc[0]

    cols_to_interpolate = ['weighted_mean', 'std_dev', 'std_error', 'head_count']

    for col in cols_to_interpolate:
        if col in full_df.columns:
            if 2010 in full_df.index and not pd.isna(full_df.loc[2010, col]):
                val_2010 = full_df.loc[2010, col]
                full_df.loc[start_year:2010, col] = full_df.loc[start_year:2010, col].fillna(val_2010)

            full_df[col] = full_df[col].interpolate(method='linear', limit_area='inside')

    return full_df.reset_index()

# Step 1: Perform interpolation on the long-format data
interpolated_long_df = df.groupby(['GEOID', 'species'], group_keys=False).apply(interpolate_group).reset_index(drop=True)

# Step 2: Pivot the interpolated data to achieve the wide format
interpolated_wide_df = interpolated_long_df.pivot_table(
    index=['GEOID', 'year'],
    columns='species',
    values=['weighted_mean', 'std_dev', 'std_error', 'head_count', 'area_usable_km2']
).reset_index()

# Step 3: Flatten the multi-level columns
interpolated_wide_df.columns = [f'{col[0]}_{col[1]}' if col[0] in ['weighted_mean', 'std_dev', 'std_error', 'head_count', 'area_usable_km2'] else col[0] for col in interpolated_wide_df.columns]

# Step 4: Add the total_area_km2 column to the wide DataFrame
# This will now work correctly after standardizing the GEOID columns
interpolated_wide_df['total_area_km2'] = interpolated_wide_df['GEOID'].map(county_area_map)

# Reorder columns for readability
cols = ['GEOID', 'year', 'total_area_km2'] + [col for col in interpolated_wide_df.columns if col not in ['GEOID', 'year', 'total_area_km2']]
interpolated_wide_df = interpolated_wide_df[cols]

# Save the final result
interpolated_wide_df.to_csv(output_file, index=False)
print(f"Interpolated wide-format data saved to {output_file} at {datetime.now().strftime('%I:%M %p CDT on %B %d, %Y')}")

Next two code boloks combines the  a) livestock dataset merge b) LE and Livestock datset

In [1]:
import pandas as pd
from pathlib import Path

# ─────────── CONFIG ───────────
DATA_DIR      = Path("/Users/faizahmad/Desktop/NewLiv/data/")
LIFEEX_CSV    = DATA_DIR / "final_dataset_103_features 1.csv"
LIV_PREPPED   = DATA_DIR / "interpolated_livestock_wide_engineered.csv"
OUTPUT_CSV    = DATA_DIR / "combined_final_livestock_wide_engineered_with_LifeEx.csv"
# ──────────────────────────────

# 1️⃣  Load LifeEX (ensure fips is string, zero‑padded)
lifeex = pd.read_csv(LIFEEX_CSV, dtype={"fips": str})
lifeex["fips"] = lifeex["fips"].str.zfill(5)
lifeex["year"] = lifeex["year"].astype(int)

# 2️⃣  Load livestock (rename GEOID → fips, zero‑pad)
liv = pd.read_csv(LIV_PREPPED, dtype={"GEOID": str})
liv["GEOID"] = liv["GEOID"].str.zfill(5)
liv = liv.rename(columns={"GEOID": "fips"})
liv["year"]  = liv["year"].astype(int)

# 3️⃣  Full outer join on fips & year
combined = pd.merge(
    lifeex,
    liv,
    on=["fips", "year"],
    how="inner",
    validate="one_to_one"
)

# 4️⃣  Save final dataset
combined.to_csv(OUTPUT_CSV, index=False)
print("✅ Combined dataset ready for ML/DL:", OUTPUT_CSV)


✅ Combined dataset ready for ML/DL: /Users/faizahmad/Desktop/NewLiv/data/combined_final_livestock_wide_engineered_with_LifeEx.csv


A total of 95 columns in our combined file before multi-colinearity cleaning

In [4]:
import pandas as pd
df3=pd.read_csv('/Users/faizahmad/Desktop/grok_live/combined_fina_2_df_Live_life_fixed.csv')
print(df3.head())
print(df3.info())
print(df3.describe())

   fips  year  MeanLifeExpectency  count_buffalo  count_cattle  count_chicken  \
0  1001  2003           74.628765           35.0          35.0           35.0   
1  1003  2003           76.661419           76.0          76.0           76.0   
2  1005  2003           74.047811           47.0          47.0           47.0   
3  1007  2003           73.057987           34.0          34.0           34.0   
4  1009  2003           75.053119           40.0          40.0           40.0   

   count_duck  count_goat  count_horse  count_pig  ...  growth_rate_hc_goat  \
0        35.0        35.0         35.0       35.0  ...                  NaN   
1        76.0        76.0         76.0       76.0  ...                  NaN   
2        47.0        47.0         47.0       47.0  ...                  NaN   
3        34.0        34.0         34.0       34.0  ...                  NaN   
4        40.0        40.0         40.0       40.0  ...                  NaN   

   growth_rate_wm_horse  growth_rate_h

/Users/faizahmad/miniforge3/envs/geo_ml/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/Users/faizahmad/miniforge3/envs/geo_ml/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/Users/faizahmad/miniforge3/envs/geo_ml/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/Users/faizahmad/miniforge3/envs/geo_ml/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


b) 

In [2]:
import pandas as pd
from pathlib import Path

# ─────────── CONFIG ───────────
DATA_DIR      = Path("/Users/faizahmad/Desktop/NewLiv/data/")
LIFEEX_CSV    = DATA_DIR / "df_livestock_with_counts.csv"
LIV_PREPPED   = DATA_DIR / "ML ready livestock_engineered_lifeexpectancy_final_df.csv"
OUTPUT_CSV    = DATA_DIR / "combined_fina_2_df_Live_life.csv"
# ──────────────────────────────

# 1️⃣  Load LifeEX (ensure fips is string, zero‑padded)
lifeex = pd.read_csv(LIFEEX_CSV, dtype={"fips": str})
lifeex["fips"] = lifeex["fips"].str.zfill(5)
lifeex["year"] = lifeex["year"].astype(int)

# 2️⃣  Load livestock (rename GEOID → fips, zero‑pad)
liv = pd.read_csv(LIV_PREPPED, dtype={"fips": str})
liv["fips"] = liv["fips"].str.zfill(5)
liv["year"]  = liv["year"].astype(int)

# 3️⃣  Full outer join on fips & year
combined = pd.merge(
    lifeex,
    liv,
    on=["fips", "year"],
    how="inner",
    validate="one_to_one"
)

# 4️⃣  Save final dataset
combined.to_csv(OUTPUT_CSV, index=False)
print("✅ Combined dataset ready for ML/DL:", OUTPUT_CSV)


✅ Combined dataset ready for ML/DL: /Users/faizahmad/Desktop/NewLiv/data/combined_fina_2_df_Live_life.csv


Feature dropping for robust ML - this generates our cleanned df

In [ ]:
import pandas as pd
import numpy as np

# Load your dataset
df = pd.read_csv("/Users/faizahmad/Desktop/grok_live/ML ready livestock_engineered_lifeexpectancy_final_df.csv")

# Remove duplicate MeanLifeExpectency columns
life_cols = [col for col in df.columns if "MeanLifeExpectency" in col]
if len(life_cols) > 1:
    df = df.drop(columns=life_cols[1:])
df = df.rename(columns={life_cols[0]: "MeanLifeExpectency"})

# Exclude target and identifiers
features = df.drop(columns=['fips', 'year', 'MeanLifeExpectency'])

# Identify features to drop
missing = features.isnull().mean()
high_missing = missing[missing > 0.3].index.tolist()

low_variance = features.loc[:, features.nunique() <= 1].columns.tolist()

corr_matrix = features.corr().abs()
upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
high_corr = [col for col in upper_triangle.columns if any(upper_triangle[col] > 0.95)]

# Combine and drop
features_to_drop = set(high_missing + low_variance + high_corr)
df_cleaned = df.drop(columns=features_to_drop)

# Save cleaned file
df_cleaned.to_csv("cleaned_combined_life_df.csv", index=False)

# Optional: Save dropped columns with reasons
dropped = pd.DataFrame({'Feature': list(features_to_drop)})
dropped['Reason'] = dropped['Feature'].apply(
    lambda x: 'High Missingness (>30%)' if x in high_missing else (
        'Low Variance' if x in low_variance else 'Highly Correlated (>0.95)'
    )
)
dropped.to_csv("dropped_features_with_reasons.csv", index=False)


In [2]:
df = pd.read_csv("/Users/faizahmad/Desktop/grok_live/cleaned_combined_life_df.csv")


In [19]:
df

,fips,year,MeanLifeExpectency,count_buffalo,count_cattle,mean_buffalo,mean_cattle,mean_chicken,mean_duck,mean_goat,...,std_error_sheep,usable_area_percent_buffalo,growth_rate_wm_buffalo,growth_rate_wm_cattle,growth_rate_wm_chicken,growth_rate_wm_duck,growth_rate_wm_goat,growth_rate_wm_horse,growth_rate_wm_pig,growth_rate_wm_sheep
0,01001,2003,74.628765,35.0,35.0,65.904788,860.999684,1423.557155,8.388511,41.508960,...,0.671143,36.517602,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01003,2003,76.661419,76.0,76.0,0.000000,311.835759,28.862582,2.434907,24.505563,...,1.197483,27.840037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01005,2003,74.047811,47.0,47.0,53.621798,512.685904,103001.408125,9.907964,16.115635,...,0.515592,38.256086,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01007,2003,73.057987,34.0,34.0,14.848961,307.661134,2677.674651,2.071313,16.036272,...,0.559568,18.238589,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01009,2003,75.053119,40.0,40.0,25.794029,1477.174746,435816.552930,12.237306,56.636708,...,0.753218,78.164150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52156,56037,2019,78.018892,0.0,471.0,0.588964,13.908474,0.290212,0.051913,1.800577,...,0.200563,0.954676,0.0,-48.028656,-48.467522,0.0,-48.261210,0.0,-47.879305,-48.183869
52157,56039,2019,85.638149,0.0,202.0,0.187142,4.416270,0.289818,4.623372,0.053630,...,0.140562,6.903559,0.0,-47.880597,-48.476390,0.0,-48.020152,0.0,-47.504697,-48.061876
52158,56041,2019,77.521691,0.0,104.0,1.209951,84.895367,2.530504,0.752106,1.614481,...,2.751675,6.837231,0.0,-48.018224,-48.526783,0.0,-48.109447,0.0,-47.778035,-48.146817
52159,56043,2019,78.137158,0.0,127.0,1.669329,62.097799,2.385102,6.604897,1.493581,...,1.848628,16.175882,0.0,-47.934240,-48.503145,0.0,-48.014093,0.0,-47.819189,-48.067461


In [23]:
import pandas as pd

# Assuming 'df' is already a pandas DataFrame loaded from a CSV
# Example: df = pd.read_csv('your_file.csv')

# 1. Check data statistics for numerical columns
print("Descriptive Statistics:")
print(df.describe())
print("\n")

# 2. Display information about the DataFrame (including data types and non-null counts)
print("DataFrame Info:")
df.info()
print("\n")

# 3. Display the first row of the DataFrame
print("First Row:")
print(df.iloc[0]) # or df.head(1)
print("\n")

# 4. Display all column headers (claims head)
print("Column Headers:")
print(df.columns.tolist())

Descriptive Statistics:
               year  MeanLifeExpectency  count_buffalo  count_cattle  \
count  52161.000000        52161.000000   52161.000000  52161.000000   
mean    2011.000575           77.169712      30.923889     52.464485   
std        4.899163            2.448107      49.011731     58.658653   
min     2003.000000           65.176273       0.000000      1.000000   
25%     2007.000000           75.492520       0.000000     28.000000   
50%     2011.000000           77.276202      22.200000     37.000000   
75%     2015.000000           78.838345      39.000000     51.000000   
max     2019.000000           92.253858     807.000000    807.000000   

       mean_buffalo   mean_cattle  mean_chicken     mean_duck     mean_goat  \
count  52161.000000  52161.000000  5.216100e+04  52161.000000  52161.000000   
mean      10.069839    838.721267  2.695304e+04    104.060398     28.142092   
std       27.137042    991.635482  9.691533e+04   1776.773154     48.940265   
min        

/Users/faizahmad/miniforge3/envs/geo_ml/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/Users/faizahmad/miniforge3/envs/geo_ml/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


In a separte notebook ML codes are provided